In [1]:
import re

def load_map_and_commands():
    with open('input22.txt') as csvfile:
        map_and_commands = [[c for c in line] for line in csvfile.readlines()]
        max_monkey_map_row_size = max(len(row[:-1]) for row in map_and_commands[:-2])
        monkey_map = [row[:-1] + [' ']*(max_monkey_map_row_size-len(row[:-1])) for row in map_and_commands[:-2]]
    commands = [(int(x), y) for x, y in zip(re.findall('(\d+)', ''.join(map_and_commands[-1])), re.findall('([R|L])', ''.join(map_and_commands[-1])) + ['E'])]
    return monkey_map, commands

In [2]:
def find_box(box, length_face_edge, i, j):
    return box[(i//length_face_edge)%len(box)][(j//length_face_edge)%len(box[0])]

In [3]:
def find_box_coordinates(box, length_face_edge, box_number):
    for i in range(4):
        for j in range(4):
            if find_box(box, length_face_edge, i*length_face_edge, j*length_face_edge) == box_number:
                return [i*length_face_edge, j*length_face_edge]

In [4]:
def update_table(table, i, direction):
    clockwise_directions = ['R', 'D', 'L', 'U']
    direction_idx = clockwise_directions.index(direction)
    first_option = clockwise_directions[(direction_idx-1) % len(clockwise_directions)]
    second_option = clockwise_directions[(direction_idx+1) % len(clockwise_directions)]
   
    if table[i][direction][1] == 0:
        first_option_quadrant, second_option_quadrant = table[i][first_option], table[i][second_option]
        if first_option_quadrant[1] != 0:
            new_direction = clockwise_directions[(direction_idx - first_option_quadrant[0]//90) % len(clockwise_directions)]
            direction_first_option_quadrant = table[first_option_quadrant[1]][new_direction]
            if direction_first_option_quadrant[1] != 0:
                table[i][direction] = ((first_option_quadrant[0]+direction_first_option_quadrant[0]+90)%360, direction_first_option_quadrant[1])
        elif second_option_quadrant[1] != 0:
            new_direction = clockwise_directions[(direction_idx - second_option_quadrant[0]//90) % len(clockwise_directions)]
            direction_second_option_quadrant = table[second_option_quadrant[1]][new_direction]
            if direction_second_option_quadrant[1] != 0:
                table[i][direction] = ((second_option_quadrant[0]+direction_second_option_quadrant[0]-90%360), direction_second_option_quadrant[1])
                
    return table

In [5]:
def check_collision(monkey_map, next_position, next_direction):
    cell = monkey_map[next_position[0] % len(monkey_map)][next_position[1] % len(monkey_map[0])]
    if  cell == '#':
        return True
    #elif cell == '.':
    elif cell == '.' or cell in ['U', 'R', 'D', 'L']:
        monkey_map[next_position[0] % len(monkey_map)][next_position[1] % len(monkey_map[0])] = next_direction
        return False
    else:
        return None

In [6]:
def next_direction(current_direction, angle):
    clockwise_directions = ['R', 'D', 'L', 'U']
    return clockwise_directions[(clockwise_directions.index(current_direction) - angle//90) % len(clockwise_directions)]

In [7]:
def walk_one_step_3d(current_position, current_direction, table = None, box = None, length_face_edge = None):
    next_position = current_position[:]
    if current_direction in ['D', 'U']:
        next_position[0] += 1 if current_direction == 'D' else -1
    else:
        next_position[1] += 1 if current_direction == 'R' else -1

    possible_next_box = find_box(box, length_face_edge, next_position[0], next_position[1])
    current_box = find_box(box, length_face_edge, current_position[0], current_position[1])

    if possible_next_box != current_box:
        angle, real_next_box = table[current_box][current_direction]
        
        current_box_coordinates = find_box_coordinates(box, length_face_edge, current_box)
        next_box_coordinates = find_box_coordinates(box, length_face_edge, real_next_box)
   
        y_min, y_max = next_box_coordinates[0], next_box_coordinates[0]+length_face_edge-1
        x_min, x_max = next_box_coordinates[1], next_box_coordinates[1]+length_face_edge-1
        y_relative, x_relative = (current_position[0] % length_face_edge), (current_position[1] % length_face_edge)
        y_absolute, x_absolute = y_min + y_relative, x_min + x_relative
        
        if angle == 0:
            if current_direction == 'U': 
                next_position = [y_max, x_absolute]
            elif current_direction == 'D':
                next_position = [y_min, x_absolute]
            elif current_direction == 'R':
                next_position = [y_absolute, x_min]
            elif current_direction == 'L':
                next_position = [y_absolute, x_max]
        elif angle in [180, -180]:
            if current_direction == 'U': 
                next_position = [y_min, x_max-x_relative]
            elif current_direction == 'D':
                next_position = [y_max, x_max-x_relative]
            elif current_direction == 'R':
                next_position = [y_max-y_relative, x_max]
            elif current_direction == 'L':
                next_position = [y_max-y_relative, x_min]
        elif angle in [90, -270]:
            if current_direction == 'U': 
                next_position = [y_max-x_relative, x_max]
            elif current_direction == 'D':
                next_position = [y_max-x_relative, x_min]
            elif current_direction == 'R':
                next_position = [y_max, x_min+y_relative]
            elif current_direction == 'L':
                next_position = [y_min, x_min+y_relative]
        elif angle in [-90, 270]:
            if current_direction == 'U': 
                next_position = [y_min+x_relative, x_min]
            elif current_direction == 'D':
                next_position = [y_min+x_relative, x_max]
            elif current_direction == 'R':
                next_position = [y_min, x_max-y_relative]
            elif current_direction == 'L':
                next_position = [y_max, x_max-y_relative]
        return next_position, next_direction(current_direction, angle)
    else:
        return next_position, current_direction

In [8]:
def walk_one_step_2d(current_position, current_direction):
    next_position = current_position[:]
    if current_direction in ['D', 'U']:
        next_position[0] += 1 if current_direction == 'D' else -1
    else:
        next_position[1] += 1 if current_direction == 'R' else -1
    return next_position, current_direction

In [9]:
def walk_one_step(current_position, current_direction, table = None, box = None, length_face_edge = None):
    if table is not None:
        return walk_one_step_3d(current_position, current_direction, table, box, length_face_edge)
    else:
        return walk_one_step_2d(current_position, current_direction)

In [10]:
def one_walk(monkey_map, current_position, current_direction, n_steps, table = None, box = None, length_face_edge = None):
    for _ in range(n_steps):
        next_position, next_direction = walk_one_step(current_position, current_direction, table, box, length_face_edge)
        while (check_collision(monkey_map, next_position, next_direction) is None):
            next_position, next_direction = walk_one_step(next_position, next_direction, table, box, length_face_edge)
        if check_collision(monkey_map, next_position, next_direction):
            return current_position, current_direction
        current_position, current_direction = next_position, next_direction
    return current_position, current_direction

In [11]:
def walk(monkey_map, commands, table = None, box = None, length_face_edge = None):
    current_position = [0, monkey_map[0].index('.')]
    current_direction = 'R'

    for command in commands:
        current_position, current_direction = one_walk(monkey_map, current_position, current_direction, command[0], table, box, length_face_edge)
        current_direction = next_direction(current_direction, -90 if command[1] == 'R' else +90 if command[1] == 'L' else 0)
    
    return current_position, current_direction

In [12]:
def password(monkey_map, position, direction):
    map_facing_to_int = {'R': 0, 'D': 1, 'L': 2, 'U': 3}
    return 1_000 * ((position[0] % len(monkey_map)) + 1) + 4*((position[1] % len(monkey_map[0])) + 1) + (map_facing_to_int[direction])

Part 01

In [13]:
monkey_map, commands = load_map_and_commands()
position, direction = walk(monkey_map, commands)
answer = password(monkey_map, position, direction)
answer

3590

Part 02

In [14]:
monkey_map, commands = load_map_and_commands()

length_face_edge = int((sum(row.count('#') + row.count('.') for row in monkey_map) / 6)**0.5)
row_faces, column_faces = (len(monkey_map) // length_face_edge, len(monkey_map[0]) // length_face_edge) if len(monkey_map) % 3 else (len(monkey_map) // length_face_edge, len(monkey_map[0]) // length_face_edge)

quadrant = 1
box = [[None for _ in range(4)] for _ in range(4)]
for i in range(4):
    for j in range(4):
        if i < row_faces and j < column_faces and monkey_map[i*length_face_edge][j*length_face_edge] != ' ':
            box[i][j] = quadrant
            quadrant+=1
        else:
            box[i][j] = 0
            
table = {}
for i in range(4):
    for j in range(4):
        if box[i][j] != 0:
            table[box[i][j]] = {'U': (0, box[(i-1) % len(box)][j]), \
                                'D': (0, box[(i+1) % len(box)][j]), \
                                'L': (0, box[i][(j-1) % len(box[i])]), \
                                'R': (0, box[i][(j+1) % len(box[i])])}
            
for _ in range(3):
    for i in range(1, quadrant):
        for direction in ['U', 'R', 'D', 'L']:
            table = update_table(table, i, direction)

In [15]:
position, direction = walk(monkey_map, commands, table, box, length_face_edge)
answer = password(monkey_map, position, direction)
answer

86382